In [ ]:
import os

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from transformers import BertJapaneseTokenizer, BertModel
from dotenv import load_dotenv
import torch
import pandas as pd

In [ ]:
ENABLE_INDEX=True
IS_TEST=False

In [ ]:
load_dotenv(dotenv_path='.local.env')

In [ ]:
class SentenceBertJapanese:
    """
    https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
    """

    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)


MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"  # <- v2です。
model = SentenceBertJapanese(MODEL_NAME)

sentences = ["暴走したAI", "暴走した人工知能"]
sentence_embeddings = model.encode(sentences, batch_size=8)

print("Sentence embeddings:", sentence_embeddings)

In [ ]:
# 何かしら素敵なデータをCSVで用意しておく
df = pd.read_csv('resources/data.csv', engine='python')

In [ ]:
titles = df['title'].to_list()
print(len(titles))

In [ ]:
# 動作確認用
if IS_TEST:
    titles = ["新型コロナワクチン予防接種について", "胃ガンの手術", '三歳の子供が熱']

In [ ]:
sentence_embeddings = model.encode(titles, batch_size=8)

In [ ]:
ES_URL = os.environ.get("ES_URL")
ES_USER = os.environ.get("ES_USER")
ES_PASS = os.environ.get("ES_PASS")
INDEX_NAME = os.environ.get("ES_INDEX_NAME")
print(ES_URL)
print(ES_USER)
print(ES_PASS)

es = Elasticsearch(ES_URL, basic_auth=(ES_USER, ES_PASS))
es.cat.indices()

In [ ]:
actions = [{
    "_id": _id,
    "_source": {
        "title": title,
        "vector": embedding,
    },
} for _id, (title, embedding) in enumerate(zip(titles, sentence_embeddings.tolist()))]

In [ ]:
if ENABLE_INDEX:
    success, errors = bulk(es, actions, index=INDEX_NAME, refresh=True)

print(success)
print(errors)

In [ ]:
search_title = '三歳児の高熱'
search_embeddings = model.encode([search_title], batch_size=8)

query = {
    "query": {
        "term": {
            "title": search_title
        }
    },
    "knn": {
        "field": "vector",
        "query_vector": search_embeddings.tolist()[0],
        "k": 10,
        "num_candidates": 50
    },
    "rank": {
        "rrf": {
            "window_size": 50,
            "rank_constant": 20
        }
    }
}

In [ ]:

result = es.search(index=INDEX_NAME, body=query, size=10)
# 検索結果からドキュメントの内容のみ表示
for document in result["hits"]["hits"]:
    print(document["_source"]['title'])